In [21]:
!pip -q install Flask==3.0.3 python-dotenv==1.0.1 requests==2.32.3 pandas==2.2.2 pyngrok==7.2.0

In [32]:
!pip -q uninstall -y python-telegram-bot httpx httpcore h2 hpack h11
!pip -q install "httpx[http2]==0.28.1" "python-telegram-bot==21.6" pandas==2.2.2 requests==2.32.3 nest_asyncio==1.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 652.1/652.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00


In [22]:
import os

# ==== WA Cloud API (WAJIB) ====
os.environ["ACCESS_TOKEN"]    = "EAAVO6xXnyqIBPOjbHOlxAMIiQtJm6UHK82dRxmZAxGeLHUk6i7bpkbBmMoOBsVZCL4Kgy5jCbToAipaQriT0EZCSDnZAlz14RkIZAw5agHDTk77ig2l4GYyVQDv2pZAphZBaOjeDESltI4MKY9uAz42qUUd2ZBZC9yyzh8qm0L1OyANGbKJff6k8T5ceDmZBkKmbky7CpvatArWipC77SfsPqCbffzCJv58tNAntgr"  # long-lived token
os.environ["PHONE_NUMBER_ID"] = "781114708412446"
os.environ["VERIFY_TOKEN"]    = "AIDSPIRE_VERIFY_TOKEN"

# ==== CSV (WAJIB) ====
os.environ["CSV_PATH"] = "/myDrive/risk_by_kelurahan.csv"

# ==== CARTO (UNTUK LOKASI → BQ Stored Proc via CARTO) ====
# Endpoint yang menerima body JSON: {q: "CALL ...", queryParameters: {...}}
os.environ["CARTO_SQL_API_URL"] = "https://gcp-us-east1.api.carto.com/v3/sql/carto_dw/query"


# Jika endpoint butuh auth (Bearer)
os.environ["CARTO_API_KEY"]     = "eyJhIjoiYWNfa2k1bW04ZnIiLCJqdGkiOiI4ZGM0MjA3NSJ9eyJhIjoiYWNfa2k1bW04ZnIiLCJqdGkiOiI4ZGM0MjA3NSJ9"

# Nama prosedur persis (termasuk backtick)
os.environ["CARTO_CALL_PROC"]   = "`carto-dw-ac-ki5mm8fr.workflows_temp_gandes_hackjkt_968c080b.wfproc_api_55d3874212100e15`"


In [ ]:

import os
os.environ["TELEGRAM_BOT_TOKEN"] = "EAAVO6xXnyqIBPOjbHOlxAMIiQtJm6UHK82dRxmZAxGeLHUk6i7bpkbBmMoOBsVZCL4Kgy5jCbToAipaQriT0EZCSDnZAlz14RkIZAw5agHDTk77ig2l4GYyVQDv2pZAphZBaOjeDESltI4MKY9uAz42qUUd2ZBZC9yyzh8qm0L1OyANGbKJff6k8T5ceDmZBkKmbky7CpvatArWipC77SfsPqCbffzCJv58tNAntgr"

# CARTO (pakai saat user kirim lokasi/pin)
os.environ["CARTO_SQL_API_URL"] = "https://<YOUR-CARTO-ENDPOINT>/sql/execute"
os.environ["CARTO_CALL_PROC"]   = "`carto-dw-ac-ki5mm8fr.workflows_temp_gandes_hackjkt_968c080b.wfproc_api_55d3874212100e15`"
os.environ["CARTO_API_KEY"]     = ""  # isi token kalau endpoint butuh Authorization Bearer

# CSV: upload dulu ke /content, atau ganti path ke Drive
os.environ["CSV_PATH"] = "/content/risk_by_kelurahan.csv"
print("CSV_PATH =", os.environ["CSV_PATH"])


In [23]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import os
os.environ["CSV_PATH"] = "/content/drive/MyDrive/Aidspire/risk_by_kelurahan.csv"
print("CSV_PATH =", os.environ["CSV_PATH"], "exists?", os.path.exists(os.environ["CSV_PATH"]))

CSV_PATH = /content/drive/MyDrive/Aidspire/risk_by_kelurahan.csv exists? True


In [25]:
import os, re, json, logging, requests, threading, time
import pandas as pd
from flask import Flask, request, jsonify
from pyngrok import ngrok

ACCESS_TOKEN     = os.getenv("ACCESS_TOKEN", "")
PHONE_NUMBER_ID  = os.getenv("PHONE_NUMBER_ID", "")
VERIFY_TOKEN     = os.getenv("VERIFY_TOKEN", "")
CSV_PATH         = os.getenv("CSV_PATH", "/content/drive/MyDrive/Aidspire/risk_by_kelurahan.csv")

# CARTO CALL
CARTO_SQL_API_URL = os.getenv("CARTO_SQL_API_URL", "")
CARTO_API_KEY     = os.getenv("CARTO_API_KEY", "")
CARTO_CALL_PROC   = os.getenv("CARTO_CALL_PROC", "")

logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger("aidspire-colab")

# ---------- Utils & CSV loader (robust) ----------
def normalize_name(s: str) -> str:
    import re
    return re.sub(r"\s+", " ", (s or "").strip()).casefold()

def load_csv_safely(csv_path: str) -> pd.DataFrame:
    import os
    assert os.path.exists(csv_path), f"CSV tidak ditemukan di {csv_path}. Pastikan sudah upload ke Colab."

    last_err = None
    for enc in ("utf-8", "utf-8-sig", "latin1"):
        try:
            df_ = pd.read_csv(csv_path, encoding=enc)
            break
        except Exception as e:
            last_err = e
            df_ = None
    if df_ is None:
        raise RuntimeError(f"Gagal membaca CSV. Error terakhir: {last_err}")

    cols_lower = {c.lower(): c for c in df_.columns}
    kel_candidates  = ["kelurahan", "village", "desa", "subdistrict_name"]
    risk_candidates = ["flood_class", "floodclass", "risk_class", "kelas_risiko"]

    def pick(cands, pool):
        for k in cands:
            if k in pool: return pool[k]
        return None

    kel_col  = pick(kel_candidates, cols_lower)
    risk_col = pick(risk_candidates, cols_lower)
    if not kel_col or not risk_col:
        raise AssertionError(f"CSV wajib punya kolom kelurahan & flood_class. Kolom ada: {list(df_.columns)}")

    df_["k_norm"] = df_[kel_col].astype(str).map(normalize_name)
    for col in ["count_faskes","count_pengungsian","count_genangan","aid_priority"]:
        if col not in df_.columns:
            df_[col] = 0 if col != "aid_priority" else ""

    df_.rename(columns={kel_col:"kelurahan", risk_col:"flood_class"}, inplace=True)
    logger.info("CSV loaded: rows=%s, sample=%s", len(df_), df_[["kelurahan","flood_class"]].head(2).to_dict("records"))
    return df_

df = load_csv_safely(CSV_PATH)

SESSIONS = {}  # phone -> {"kelurahan": "Nama Kelurahan"}

RISK_TO_STATUS = {
    "risiko sangat rendah": "Normal",
    "risiko rendah": "Waspada",
    "risiko medium": "Siaga",
    "risiko tinggi": "Awas",
    "sangat tinggi": "Awas",
}

EMERGENCY_INFO = (
    "📢 Informasi dan Layanan Darurat Banjir Jakarta\n"
    "Pantau info banjir terkini di 👉 pantaubanjir.jakarta.go.id\n"
    "Ikuti media sosial resmi BPBD DKI Jakarta:\n"
    "Instagram: @bpbddkijakarta\n"
    "Facebook: BPBD DKI Jakarta\n"
    "Laporkan banjir di sekitar rumahmu lewat aplikasi JAKI.\n"
    "Butuh bantuan darurat? Hubungi Call Center 112 (gratis & aktif 24 jam)."
)

HELP_TEXT = (
    "Halo 👋\n"
    "Untuk melihat informasi risiko banjir di daerah Anda, silakan ketik nama kelurahan atau lokasi Anda.\n\n"
    "Contoh: Kelurahan Cempaka Putih"
)

NOT_FOUND_TEXT = (
    "Lokasi/kelurahan tidak ditemukan.\n"
    "Silakan pilih salah satu:\n"
    "🔄 Masukkan Ulang Lokasi\n"
    "❌ Akhiri Chat"
)

END_TEXT = (
    "Terima kasih sudah chat hari ini 😊\n"
    "Jaga kesehatan dan tetap waspada terhadap banjir ya.\n"
    "Sampai ketemu lagi! 👋\n\n"
    f"{EMERGENCY_INFO}"
)

def lookup_by_kelurahan(nama_kelurahan: str):
    key = normalize_name(nama_kelurahan)
    row = df.loc[df["k_norm"] == key]
    return row.iloc[0].to_dict() if not row.empty else None

def msg_step1_menu(nama_kelurahan: str, flood_class: str, with_facilities: bool=True):
    header = (
        f"Anda berada di Kelurahan {nama_kelurahan} dengan tingkat risiko banjir: {flood_class}.\n\n"
        f"{EMERGENCY_INFO}\n\n"
        "Pilih informasi yang Anda butuhkan:\n"
        "- Persiapan Banjir → apa saja yang perlu dilakukan sebelum banjir.\n"
        "- Waspada Banjir → langkah saat ada tanda-tanda banjir.\n"
        "- Saat Banjir → hal yang harus dan tidak boleh dilakukan.\n"
        "- Sesudah Banjir → panduan pemulihan setelah banjir surut.\n"
    )
    if with_facilities:
        header += "- Daftar fasilitas kesehatan terdekat ->\n- Lokasi pengungsian terdekat -->\n"
    header += "- Akhiri Chat → keluar dari percakapan."
    return header

def msg_persiapan():
    return (
        "📘 Panduan Kesiapsiagaan Banjir untuk Masyarakat\n\n"
        "✅ Saat belum ada banjir:\n"
        "• Siapkan Tas Siaga (dokumen, pakaian 3 hari, obat, selimut, makanan ringan, powerbank, uang, senter, masker, peluit).\n"
        "• Kenali zona banjir, jalur evakuasi, lokasi pengungsian; simpan 112.\n"
        "• Bagi peran keluarga (listrik/kompor, tas siaga, titik kumpul).\n"
        "• Jaga lingkungan (bersihkan got, jangan buang sampah ke sungai).\n"
        "• Pahami peringatan dini (SMS/sirine/RT/RW/BPBD).\n\n"
        "❌ Hindari: simpan dokumen di tempat rendah, bangun di bantaran, buang sampah ke got/sungai, menutup saluran air, mengabaikan peringatan dini."
    )

def msg_waspada():
    return (
        "⚠️ Jika sudah Waspada Banjir:\n"
        "1) Matikan listrik & kompor\n2) Siapkan Tas Siaga\n3) Pindahkan barang berharga ke tempat tinggi\n"
        "4) Ikuti info resmi BPBD\n5) Ikuti petugas/RT/RW & tetap tenang"
    )

def msg_saat_banjir():
    return (
        "🌊 Saat Banjir Terjadi\n"
        "✅ Lakukan: mengungsi via jalur evakuasi; dahulukan kelompok rentan; waspada arus/got; ikuti info resmi; jaga keamanan; jaga kebersihan & kesehatan.\n"
        "❌ Hindari: berkendara di jalan banjir; berjalan di arus deras tanpa alat bantu.\n\n"
        "Ketik *Menu* untuk kembali, atau *Akhiri* untuk keluar."
    )

def msg_sesudah_banjir():
    return (
        "✅ Setelah Banjir Surut\n"
        "• Pakai alas kaki • Pastikan rumah aman • Cari info bantuan • Bersihkan lumpur • Waspada hewan • Cek listrik • Sterilkan rumah • Waspadai longsor • Perbaiki saluran air • Ke faskes bila ada keluhan\n\n"
        "Ketik *Menu* untuk kembali, atau *Akhiri* untuk keluar."
    )

def msg_faskes(kel: str, row: dict):
    n = int(row.get("count_faskes", 0))
    if n <= 0:
        return f"Data fasilitas kesehatan untuk Kelurahan {kel} belum tersedia.\nKetik *Menu* untuk kembali, atau *Akhiri* untuk keluar."
    return f"Jumlah fasilitas kesehatan terdekat dari Kelurahan {kel}: {n} lokasi.\nKetik *Menu* untuk kembali, atau *Akhiri* untuk keluar."

def msg_pengungsian(kel: str, row: dict):
    n = int(row.get("count_pengungsian", 0))
    if n <= 0:
        return f"Data lokasi pengungsian untuk Kelurahan {kel} belum tersedia.\nKetik *Menu* untuk kembali, atau *Akhiri* untuk keluar."
    return f"Jumlah lokasi pengungsian terdekat dari Kelurahan {kel}: {n} lokasi.\nKetik *Menu* untuk kembali, atau *Akhiri* untuk keluar."

# ----- format jarak (meter → m / km)
def fmt_dist_m(m):
    try:
        m = float(m)
        return f"{m:.0f} m" if m < 1000 else f"{m/1000:.1f} km"
    except:
        return "-"


In [26]:

# ----- WhatsApp helpers
def wa_send_text(to_number: str, body: str, preview_url: bool = False, max_len: int = 4096):
    url = f"https://graph.facebook.com/v20.0/{PHONE_NUMBER_ID}/messages"
    headers = {"Authorization": f"Bearer {ACCESS_TOKEN}", "Content-Type": "application/json"}

    def _post_one(chunk: str):
        payload = {"messaging_product": "whatsapp","to": to_number,"type": "text","text": {"preview_url": preview_url, "body": chunk}}
        for attempt in range(3):
            try:
                resp = requests.post(url, headers=headers, json=payload, timeout=15)
                status = resp.status_code
                try: data = resp.json()
                except Exception: data = {"text": resp.text}
                logger.info("WA send attempt=%s status=%s resp=%s", attempt+1, status, data)
                if status in (429,500,502,503,504):
                    time.sleep(1.5 * (attempt + 1)); continue
                return status, data
            except Exception as e:
                logger.warning("WA send exception (attempt %s): %s", attempt+1, e)
                time.sleep(1.0 * (attempt + 1))
        return 599, {"error": "max retries reached"}

    chunks = [body[i:i+max_len] for i in range(0, len(body), max_len)] or [""]
    last_status, last_data = None, None
    for i, ch in enumerate(chunks, 1):
        suffix = f"\n({i}/{len(chunks)})" if len(chunks) > 1 else ""
        last_status, last_data = _post_one(ch + suffix)
        if i < len(chunks): time.sleep(0.2)
    return last_status, last_data

def parse_incoming(data: dict):
    """
    Return: phone, text, mtype, loc, meta
    - text: text biasa atau hasil interactive (button/list)
    - loc: {"lat":..., "lng":...} kalau ada
    """
    try:
        entry = (data.get("entry") or [None])[0] or {}
        change = (entry.get("changes") or [None])[0] or {}
        value  = change.get("value") or {}
        messages = value.get("messages") or []
        if not messages:
            return None, None, None, None, {}
        msg = messages[0]
        mtype = msg.get("type")
        phone = msg.get("from")

        meta = {
            "message_id": msg.get("id"),
            "timestamp": msg.get("timestamp"),
            "profile_name": ((value.get("contacts") or [{}])[0].get("profile") or {}).get("name"),
            "raw_type": mtype,
        }

        text, loc = None, None
        if mtype == "text":
            text = (msg.get("text") or {}).get("body", "")
        elif mtype == "location":
            loc = {"lat": msg["location"].get("latitude"), "lng": msg["location"].get("longitude")}
        elif mtype == "interactive":
            interactive = msg.get("interactive") or {}
            itype = interactive.get("type")
            if itype == "button_reply":
                br = interactive.get("button_reply") or {}
                text = br.get("id") or br.get("title") or ""
            elif itype == "list_reply":
                lr = interactive.get("list_reply") or {}
                text = lr.get("id") or lr.get("title") or ""
            else:
                text = ""
        else:
            text = ""

        return phone, text, mtype, loc, meta
    except Exception as e:
        logger.warning("parse_incoming err: %s | head=%s", e, str(data)[:500])
        return None, None, None, None, {}

In [27]:
# ----- CARTO CALL (stored procedure + queryParameters)
def fetch_carto_info(lat: float, lng: float):
    if not CARTO_SQL_API_URL or not CARTO_CALL_PROC:
        return None
    payload = {
        "q": f"CALL {CARTO_CALL_PROC}(@longitude,@latitude)",
        "queryParameters": {"longitude": float(lng), "latitude": float(lat)}
    }
    headers = {"Content-Type": "application/json"}
    if CARTO_API_KEY:
        headers["Authorization"] = f"Bearer {CARTO_API_KEY}"
    try:
        r = requests.post(CARTO_SQL_API_URL, headers=headers, json=payload, timeout=20)
        r.raise_for_status()
        data = r.json()
        rows = data.get("rows") or []
        return rows[0] if rows else None
    except Exception as e:
        logger.warning("CARTO error: %s", e)
        return None

def msg_from_carto(row: dict) -> str:
    kel = row.get("kelurahan", "-")
    fclass = row.get("flood_class", "Risiko Rendah")
    status = RISK_TO_STATUS.get(normalize_name(fclass), "Waspada")
    base = {
        "Normal":  "ℹ️ Kondisi Normal di lokasi Anda.",
        "Waspada": "⚠️ Status Waspada di lokasi Anda.",
        "Siaga":   "🚨 Status Siaga di lokasi Anda.",
        "Awas":    "🛑 Status Awas di lokasi Anda."
    }[status]

    gd = row.get("genangan_distance_m")
    gt = row.get("genangan_tanggal", "-")
    gl = row.get("genangan_lokasi", "-")
    gw = row.get("genangan_wilayah", "-")
    gh = row.get("genangan_tinggi_meter", "-")
    gk = row.get("genangan_kategori", "-")
    pd = row.get("pengungsian_distance_m")
    pn = row.get("pengungsian_name", "-")

    ringkasan = (
        f"\n\nRingkasan lokasi:"
        f"\n• Kelurahan: {kel}"
        f"\n• Kelas risiko: {fclass} → Status: {status}"
    )

    info_genangan = ""
    try:
        if gd is not None and float(gd) <= 1000.0:
            info_genangan = (
                f"\n\n📍 Genangan terdekat {fmt_dist_m(gd)}"
                f"\n• Lokasi: {gl} ({gw})"
                f"\n• Tanggal: {gt}"
                f"\n• Tinggi ± {gh} cm — Kategori: {gk}"
            )
        elif gd is not None:
            info_genangan = f"\n\nTidak ada genangan dalam radius 1 km (terdekat {fmt_dist_m(gd)})."
    except Exception:
        pass

    info_peng = ""
    try:
        if pd is not None:
            info_peng = f"\n\n🏚️ Pengungsian terdekat: {pn} ({fmt_dist_m(pd)})"
    except Exception:
        pass

    edukasi = ""
    if status in ("Waspada","Siaga","Awas"):
        edukasi = (
            "\n\nTips singkat:\n"
            "• Amankan listrik & kompor • Siapkan Tas Siaga • Simpan dokumen penting di tempat tinggi • Ikuti info resmi BPBD"
        )

    return base + ringkasan + info_genangan + info_peng + edukasi


In [28]:

# ---------- Flask ----------
app = Flask(__name__)

@app.get("/")
def home():
    return jsonify({"ok": True, "message": "Aidspire WhatsApp Bot (Colab) aktif."})

@app.get("/webhook")
def verify():
    mode = request.args.get("hub.mode")
    token = request.args.get("hub.verify_token")
    challenge = request.args.get("hub.challenge")
    if mode == "subscribe" and token == VERIFY_TOKEN:
        return challenge, 200
    return "Forbidden", 403

@app.post("/webhook")
def webhook():
    payload = request.get_json(force=True, silent=True) or {}
    phone, text, mtype, loc, meta = parse_incoming(payload)
    logger.info("from=%s type=%s text=%r loc=%s meta=%s", phone, mtype, text, loc, meta)
    if not phone:
        return jsonify({"status":"ignored"}), 200

    # TEXT
    if mtype == "text" and text:
        tnorm = (text or "").strip().casefold()

        if any(w in tnorm for w in ("halo","hai","hello")) and "cek" not in tnorm:
            wa_send_text(phone, HELP_TEXT); return jsonify({"status":"ok"}), 200

        if tnorm in ("akhiri","akhiri chat","selesai"):
            wa_send_text(phone, END_TEXT); return jsonify({"status":"ok"}), 200

        if tnorm in ("menu","kembali","home"):
            kel = SESSIONS.get(phone, {}).get("kelurahan")
            if kel:
                row = lookup_by_kelurahan(kel)
                if row:
                    wa_send_text(phone, msg_step1_menu(row["kelurahan"], row.get("flood_class","")))
                    return jsonify({"status":"ok"}), 200
            wa_send_text(phone, HELP_TEXT); return jsonify({"status":"ok"}), 200

        if tnorm in ("persiapan banjir","persiapan"):
            wa_send_text(phone, msg_persiapan()); return jsonify({"status":"ok"}), 200
        if tnorm in ("waspada banjir","waspada"):
            wa_send_text(phone, msg_waspada()); return jsonify({"status":"ok"}), 200
        if tnorm in ("saat banjir","banjir"):
            wa_send_text(phone, msg_saat_banjir()); return jsonify({"status":"ok"}), 200
        if tnorm in ("sesudah banjir","setelah banjir","pasca banjir"):
            wa_send_text(phone, msg_sesudah_banjir()); return jsonify({"status":"ok"}), 200
        if tnorm.startswith("daftar fasilitas") or "fasilitas kesehatan" in tnorm:
            kel = SESSIONS.get(phone, {}).get("kelurahan")
            if kel:
                row = lookup_by_kelurahan(kel)
                if row:
                    wa_send_text(phone, msg_faskes(row["kelurahan"], row)); return jsonify({"status":"ok"}), 200
            wa_send_text(phone, NOT_FOUND_TEXT); return jsonify({"status":"ok"}), 200
        if "pengungsian" in tnorm:
            kel = SESSIONS.get(phone, {}).get("kelurahan")
            if kel:
                row = lookup_by_kelurahan(kel)
                if row:
                    wa_send_text(phone, msg_pengungsian(row["kelurahan"], row)); return jsonify({"status":"ok"}), 200
            wa_send_text(phone, NOT_FOUND_TEXT); return jsonify({"status":"ok"}), 200

        m = re.match(r"^\s*cek\s*:\s*(.+)$", text, flags=re.I)
        if m:
            nama = m.group(1).strip()
            row = lookup_by_kelurahan(nama)
            if row:
                SESSIONS[phone] = {"kelurahan": row["kelurahan"]}
                wa_send_text(phone, msg_step1_menu(row["kelurahan"], row.get("flood_class","")))
            else:
                wa_send_text(phone, NOT_FOUND_TEXT)
            return jsonify({"status":"ok"}), 200

        row = lookup_by_kelurahan(text)
        if row:
            SESSIONS[phone] = {"kelurahan": row["kelurahan"]}
            wa_send_text(phone, msg_step1_menu(row["kelurahan"], row.get("flood_class","")))
        else:
            wa_send_text(phone, NOT_FOUND_TEXT if len(text)>0 else HELP_TEXT)

        return jsonify({"status":"ok"}), 200

    # LOCATION → CARTO → reply
    if mtype == "location" and loc:
        lat, lng = float(loc["lat"]), float(loc["lng"])
        carto = fetch_carto_info(lat, lng)
        if carto:
            if carto.get("kelurahan"):
                SESSIONS[phone] = {"kelurahan": carto["kelurahan"]}
            wa_send_text(phone, msg_from_carto(carto))
        else:
            wa_send_text(phone, "Lokasi diterima. Untuk sementara, silakan ketik nama kelurahan.\nContoh: Kelurahan Cempaka Putih")
        return jsonify({"status":"ok"}), 200

    wa_send_text(phone, HELP_TEXT)
    return jsonify({"status":"ok"}), 200


In [30]:
from pyngrok import ngrok, conf
import os, threading, time, subprocess

NGROK_AUTHTOKEN = "31VKnY35PSUi20t94mGPAg4kIvR_3rFxGoHsgC5SQqQ9LmdyM"
conf.get_default().auth_token = NGROK_AUTHTOKEN

os.system('fuser -k 5000/tcp || true')
try:
    ngrok.kill()
except Exception:
    pass
os.system('pkill -f "ngrok" || true')

print("Jika tetap ERR_NGROK_108, buka dashboard.ngrok.com/agents dan END semua session aktif.")

def run():
    app.run(host="0.0.0.0", port=5000)
threading.Thread(target=run, daemon=True).start()
time.sleep(1)

public_tunnel = ngrok.connect(addr=5000, bind_tls=True)
PUBLIC_URL = public_tunnel.public_url
print("PUBLIC_URL =", PUBLIC_URL)
print("Callback URL (set di Meta):", f"{PUBLIC_URL}/webhook")


Jika tetap ERR_NGROK_108, buka dashboard.ngrok.com/agents dan END semua session aktif.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


PUBLIC_URL = https://ef57c57b6578.ngrok-free.app
Callback URL (set di Meta): https://ef57c57b6578.ngrok-free.app/webhook
